In [2]:
#http://www.superrugby.co.nz/Grandstand/HistoricalStandings/30

# import libraries
import requests
from bs4 import BeautifulSoup

# specify the url
url = 'http://www.superrugby.co.nz/Grandstand/HistoricalResults/'

page = requests.get(url + '2005') # 2005 Super 12
# need to get years 2006 - 2018

# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser').find("div", {"class": "content"})

In [3]:
season = soup.h2
print(season)

<h2>2005 Super 12</h2>


In [4]:
data = soup.table

In [45]:
import pandas as pd
import re

pattern = re.compile("^(Round|Semifinal).*")

table_nice = []
table_round = []

table_body = data.find('tbody')

rows = table_body.find_all('tr')

for row in rows:
    cols = row.find_all('td')
    cols_nice = [ele.text.strip() for ele in cols]
    cols_round = [x.text.strip() for x in cols if pattern.match(x.text.strip())]
    table_nice.append([ele for ele in cols_nice if ele]) # Get rid of empty values
    table_round.append([ele for ele in cols_round if ele]) # Get rid of empty values
   
df1 = pd.DataFrame(table_nice)
df2 = pd.DataFrame(table_round).fillna(method='ffill')
df = pd.concat([df1, df2], axis=1)

print(df.head(5))

         0                     1          2        3      4        0
0  Round 1                  None       None     None   None  Round 1
1   25 Feb   Highlanders v Blues    Dunedin  7:35 PM  14-30  Round 1
2   25 Feb     Waratahs v Chiefs     Sydney  9:40 PM   25-7  Round 1
3   26 Feb     Stormers v Sharks  Cape Town  5:10 AM  26-12  Round 1
4   26 Feb  Brumbies v Crusaders   Canberra  9:00 PM  32-21  Round 1


In [46]:
df.dropna(inplace = True) 
#data[1]= data[1].str.split("v", n = 1, expand = True) 
df.columns = ['date','teams','location',
                     'time','score','round']

def outcome(f):
    if f > 0:
        return 'W'
    elif f < 0:
        return 'L'
    else:
        return 'D'

df['home'] = df['teams'].str.split('v').str[0]
df['away'] = df['teams'].str.split('v').str[1]
df['hp'] = df['score'].str.split('-').str[0].astype('int') # home points
df['ap'] = df['score'].str.split('-').str[1].astype('int') # away points
df['sm'] = df['hp'] - df['ap'] # score margin
df['ho'] = [outcome(x) for x in df['sm']] # home outcome

remove_columns = ['teams','score']
df = df.drop(columns=remove_columns)

print(df.head(15))
print(df.columns)

      date                   location      time    round          home  \
1   25 Feb                    Dunedin   7:35 PM  Round 1  Highlanders    
2   25 Feb                     Sydney   9:40 PM  Round 1     Waratahs    
3   26 Feb                  Cape Town   5:10 AM  Round 1     Stormers    
4   26 Feb                   Canberra   9:00 PM  Round 1     Brumbies    
5   26 Feb                   Brisbane  11:05 PM  Round 1         Reds    
6   27 Feb               Johannesburg   4:00 AM  Round 1         Cats    
8   04 Mar        Eden Park, Auckland   7:35 PM  Round 2        Blues    
9   04 Mar                   Canberra   9:40 PM  Round 2     Brumbies    
10  05 Mar               Invercargill   5:30 PM  Round 2  Highlanders    
11  05 Mar  AMI Stadium, Christchurch   7:35 PM  Round 2    Crusaders    
12  06 Mar                     Durban   5:30 AM  Round 2       Sharks    
13  06 Mar               Johannesburg   5:30 AM  Round 2         Cats    
15  11 Mar                    Dunedin 

In [47]:
print(df.tail(15))

      date                       location     time       round          home  \
65  06 May            Eden Park, Auckland  7:35 PM    Round 11        Blues    
66  06 May                         Sydney  9:40 PM    Round 11     Waratahs    
67  07 May                        Dunedin  7:35 PM    Round 11  Highlanders    
68  07 May                       Canberra  9:40 PM    Round 11     Brumbies    
69  08 May                         Durban  1:00 AM    Round 11       Sharks    
70  08 May                      Cape Town  3:00 AM    Round 11     Stormers    
72  13 May      AMI Stadium, Christchurch  7:35 PM    Round 12    Crusaders    
73  13 May                         Sydney  9:40 PM    Round 12     Waratahs    
74  14 May  FMG Stadium Waikato, Hamilton  7:35 PM    Round 12       Chiefs    
75  14 May                       Brisbane  9:40 PM    Round 12         Reds    
76  15 May                       Pretoria  1:00 AM    Round 12        Bulls    
77  15 May                   Bloemfontei